# Create a 3 node availability group Azure Arc Enabled Daa Services Managed Instance

We can create a SQL Mi in only a few minutes (notin Azure, but running on a Kubernetes cluster)

First we want to check what the deployment will do


In [ ]:
$ResourceGroupName = 'beardarc'
$benspwd = Get-Secret -Name beardmi-benadmin-pwd 
$subscriptionid = Get-Secret subscription-id -AsPlainText
$tags = @{
    important   = 'WARNING - This resource is controlled by IaaC using bicep '
    creator     = 'Rob'
    project     = 'SQL MI Demo'
}

$date = Get-Date -Format yyyyMMddHHmmsss
$deploymentname = 'deploy_{0}_{1}' -f $ResourceGroupName, $date # name of the deployment seen in the activity log
$deploymentConfig = @{
    ResourceGroupName             = $ResourceGroupName
    Name                          = $deploymentname
    TemplateFile                  = 'sqlmi.bicep' 
    password                      = $benspwd
    subscription                  = $subscriptionid
    WhatIf                        = $true
}
New-AzResourceGroupDeployment @deploymentConfig



Getting the latest status of all resources...


Note: The result may contain false positive predictions (noise).
You can help us improve the accuracy of the result by opening an issue here: https://aka.ms/WhatIfIssues.

Resource and property changes are indicated with these symbols:
  + Create
  ~ Modify
  * Ignore

The deployment will update the following scope:

Scope: /subscriptions//resourceGroups/beardarc

  ~ Microsoft.AzureArcData/sqlManagedInstances/ben-starker [2021-03-02-preview]
    + properties.basicLoginInformation.password: "*******"

  * Microsoft.AzureArcData/dataControllers/beard-arc-dc
  * Microsoft.AzureArcData/dataControllers/beard-direct-dc
  * Microsoft.AzureArcData/sqlManagedInstances/benverbindung
  * Microsoft.ExtendedLocation/customLocations/beardarclocation
  * microsoft.insights/actiongroups/Azure Arc
  * microsoft.insights/activityLogAlerts/MI Updated or created
  * microsoft.insights/activityLogAlerts/New or CHanged MI
  * microsoft.insights/workbooks/5575c0aa-a401-467d-8511-ee787d10ebe0
  * Microsoft.

Once we are happy with that we can deploy

In [ ]:
$date = Get-Date -Format yyyyMMddHHmmsss
$deploymentname = 'deploy_{0}_{1}' -f $ResourceGroupName, $date # name of the deployment seen in the activity log
$deploymentConfig = @{
    ResourceGroupName             = $ResourceGroupName
    Name                          = $deploymentname
    TemplateFile                  = 'sqlmi.bicep' 
    subscription                  = $subscriptionid
    password                      = $benspwd
    WhatIf                        = $false
}
New-AzResourceGroupDeployment @deploymentConfig

Because I cannot get azdata to run in a dotnet interactive notebook, run `azdata arc sql endpoint list -n ben-starker` in a PowerShell window and set the endpoint to the `$SqlInstance` variable in the next code block

In [ ]:
$SqlInstance = '192.168.2.63,30666'
$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name beardmi-benadmin-user -AsPlainText), (Get-Secret -Name beardmi-benadmin-pwd))
$sql = Connect-DbaInstance -SqlInstance $SqlInstance -SqlCredential $benscreds

$PSDefaultParameterValues = @{
    "*dba*:SqlInstance" = $sql 
}

Invoke-DbaQuery -Query "Select @@ServerName as 'Which pod am I?'"

Which pod am I?


---------------


ben-starker-0


Now we have confirmed which pod is holding the SQL Instance we can copy a database backup to the pod using this code.

In [ ]:
Push-Location D:\SQLBackups
kubectl cp AdventureWorks2017.bak  ben-starker-0:/var/opt/mssql/backups -c arc-sqlmi -n arcdirect


In [ ]:
Get-DbaFile  -Path /var/opt/mssql/backups|fl
Restore-DbaDatabase  -DatabaseName AdventureWorks2017 -Path /var/opt/mssql/backups/AdventureWorks2017.bak


Filename    : /var/opt/mssql/backups\AdventureWorks2017.bak


SqlInstance : ben-starker-0


ComputerName         : 192.168.2.63


InstanceName         : MSSQLSERVER


SqlInstance          : ben-starker-0


...

                       MOVE N'AdventureWorks2017_log' TO 


                       N'/var/opt/mssql/data-log/AdventureWorks2017_log.ldf',  NOUNLOAD,  STATS = 


                       10}


RestoreDirectory     : /var/opt/mssql/data,/var/opt/mssql/data-log


WithReplace          : False


Now my database is ready :-)

Lets generate some random logins and users

In [ ]:
$logins =  'benji', 'bweissman','akamman', 'alevy', 'beardapp1', 'beardapp2', 'beardapp3', 'beardapp4', 'beardapp5', 'beardapp6', 'beardapp7', 'beardapp8', 'clemaire', 'csilva', 'fatherjack', 'gsartori', 'jamrtin', 'Reporting1', 'Reporting2', 'Reporting3', 'Reporting4', 'smelton', 'SockFactoryApp_User', 'soneill', 'sqladmin', 'Support1', 'Support2', 'Support3', 'Support4', 'Support5', 'Support6', 'tboggiano', 'thebeard', 'wdurkin','Argenis_Fernandez','MrsFatherJack','Cathrine_Wilhelmsen','Rie_Irish','Monica_Rathbun','Benji_BenWeissman','Joey_Dantoni', 'John_Morehouse', 'Bob_Facts_Pusatari','Containers_Nocentino','Andrew_Pruski','Matt_Gordon','Thomas_Grohser','Dirk_Hondong','Father_Johan_Brattas','Drew_FurggerwellyIgiveuptryingtospellit'

$secpwd = (Get-Secret -Name beardmi-demo-users-pwd)
foreach ($login in $logins) {
    New-DbaLogin -Login $login -SecurePassword $secpwd -Force
}

foreach ($login in $logins) {
        New-DbaDbUser   -Database AdventureWorks2017 -Login $login 
        Add-DbaDbRoleMember   -Database AdventureWorks2017 -Role db_owner -User $login -Confirm:$false
    
}

Repair-DbaDbOrphanUser  -Database AdventureWorks2017

ComputerName       : 192.168.2.63


InstanceName       : MSSQLSERVER


SqlInstance        : ben-starker-0


Name               : benji


LoginType          : SqlLogin


...

LoginType          : SqlLogin


AuthenticationType : Instance


State              : Existing


HasDbAccess        : True


DefaultSchema      : dbo


In [ ]:
New-DbaDatabase -Name KittenFactory
New-DbaDatabase -Name SockFactory
New-DbaDatabase -Name CarFactory
New-DbaDatabase -Name FlowerFactory
New-DbaDatabase -Name GadgetFactory
New-DbaDatabase -Name BikeFactory
New-DbaDatabase -Name BookFactory
New-DbaDatabase -Name FactoryDW
New-DbaDatabase -Name FactoryAdmin
New-DbaDatabase -Name FactoryDWAdmin
$databases = (Get-DbaDatabase  -ExcludeSystem).Name


$roles = 'db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin'

foreach ($login in $logins) {
    $DbNumber = Get-Random -Minimum 2 -Maximum $Databases.Count
    $Dbs = Get-Random $databases -Count $DbNumber
    foreach ($db in $dbs) {
        $rolenumber = Get-Random -Minimum 1 -Maximum $roles.Count
        $rolestoadd = Get-Random $roles -Count $rolenumber
        New-DbaDbUser  -Database $db -Login $login 
        Add-DbaDbRoleMember  -Database $db -Role $rolestoadd -User $login -Confirm:$false
    }
}